In [ ]:
# Imports
import argparse
import matplotlib.pyplot as plt
import numpy as np

from sharpness.dataloader import generate_synthetic_data, load_data, synthetic_f
from sharpness.transforms import apply_transform, transform_d
from sharpness.metric_list import metric_f, single_metrics
from sharpness import compute_metric_globally, compute_metric_locally, compute_all_metrics_globally, compute_all_metrics_locally
from sharpness.benchmark import visualize, heatmap_visualize

# Import low level functions:
from sharpness.dataloader import *
from sharpness.transforms import *

In [ ]:
##############################################################
def calc_min_max_while_dealing_with_nans( min_array, max_array ):
    # This is an auxiliary function for value_range_for_image_set. 
    # Input:  min_array and max_array and numpy arrays that may contain lots of NaNs.
    # Output: my_min and my_max are the min of min_array and the max of max_array, taking NaNs into account.

    ##### Calc min while dealing with NaNs #####
    min_n_valid = sum( np.isnan(min_array) == False )  # Number of valid entries    
    if min_n_valid == 0:  # all values are NaNs
        my_min = float("nan")  # If ALL values are NaNs, then manually assign NaN  
    else: # at least one value is not NaN 
        my_min = np.nanmin( min_array )  # nanmin ignores NaNs, as long at least one value is not NaN
    
    ##### Calc max while dealing with NaNs #####
    max_n_valid = sum( np.isnan(max_array) == False )
    if max_n_valid == 0:  
        my_max = float("nan")  # If ALL values are NaNs, then manually assign NaN
    else: # at least one value is not NaN 
        my_max = np.nanmax( max_array )  # nanmin ignores NaNs, as long at least one value is not NaN
    
    return( my_min, my_max )
    
##############################################################
def value_range_for_image_set( image_set_keys, image_set_dict ):
    # We often want a set of images to have the same color bar. 
    # This function calculates a suitable range of values across all images in the given set. 
    # Input: 
    #   image_set_dict is a dictionary with (img, img_string) tuples. 
    #   image_set_keys contains the keys for image_set_dict.
    
    # To calculate (cmin,cmax) across all images, first extract min and max for all individual images
    # and store in arrays, min_array, max_array.
    
    min_array = np.empty( len(image_set_keys) )
    max_array = np.empty( len(image_set_keys) )
    for image_counter, image_key in enumerate(image_set_keys):
        (img, img_string) = image_set_dict[image_key]
        min_array[image_counter] = np.min( img ) # store min value in array - to calc uniform color scale
        max_array[image_counter] = np.max( img ) # store max value in array - to calc uniform color scale

    # Calculate uniform range for color scale across all images using auxiliary function.
    cmin_uniform, cmax_uniform = calc_min_max_while_dealing_with_nans( min_array, max_array )
    return( cmin_uniform, cmax_uniform )
    
##############################################################
def apply_univariate_metrics( selected_metrics, image_dict, selected_image_keys, base_image_key, want_uniform_color_scale_per_metric, filename_prefix ):

    # Apply univariate metrics to all input images, generate heatmaps, and save image.
    # Inputs: 
    #    selected_metrics:  which metrics to apply
    #    image_dict:  dictionary with images we want to apply metrics to
    #    selected_image_keys:  keys of those images to apply metrics
    #    base_image_key:  key of the primary image that provides filename for output file
    #    want_uniform_color_scale_per_metric:  
    #         For each metric: should all heatmaps for the metric use identical color bar, or inidividual color bars?
    #
    # Output: 
    #   A single plot (on screen and saved to file) that shows
    #      First row:  all input images
    #      Second to last row:  heatmaps for all images for one metric per row
    #
    
    print('\n##############################\n     Univariate metrics\n')
    print('     min_value and max_value of each heatmap are printed underneath each panel.')
    print('     Note: color maps for heatmaps vary greatly.  Go from WHITE = min(0,min_value) to DARK GREEN = (max_value).')
    print('##############################\n\n')

    # Create a figure with subplots: columns represent transformations, rows represent metrics
    n_cols = len(selected_image_keys)
    n_rows = len(selected_metrics) + 1 # metrics, plus one row for original images
    cm = 1/2.54  # conversion factor from inches (default unit) to centimeters
    fig, axs = plt.subplots(
        ncols=n_cols, nrows= n_rows,  
        figsize= ( n_cols * 4, n_rows * 4), 
        sharex="col", sharey="row"
    )

    ######### FIRST ROW:  Plot original images 
    
    # Prep: calculate uniform cmin and cmax for first row (if desired)
    if want_uniform_color_scale_per_metric:  
        for image_counter, image_key in enumerate(selected_image_keys):
            cmin_uniform, cmax_uniform = value_range_for_image_set( selected_image_keys, image_dict )

    # Plot first row
    for image_counter, image_key in enumerate(selected_image_keys):        
        (img, img_string) = image_dict[image_key]    # Get image to print      
        if want_uniform_color_scale_per_metric:
            (cmin, cmax) = (cmin_uniform, cmax_uniform) 
        else:
            (cmin, cmax) = ( img.min(), img.max() )
        i_row = 0
        i_col = image_counter
        axs[i_row][i_col].imshow(img, clim = (np.nanmin([0.0,cmin]), cmax), cmap='gray' )
        axs[i_row][i_col].set_title(img_string)
        axs[i_row][i_col].set_xlabel(f'Min: {img.min():.2f}   Mean: {img.mean():.2f}   Max: {img.max():.2f} ')

        
    #### ALL OTHER ROWS: Plot heatmaps for all images (one metric per row)
    
    # For each metric (=each row)
    for metric_counter, my_metric in enumerate(selected_metrics):

        # For each metric: CALCULATE all heatmaps
        img_row_dict = {}  # store all heatmaps for this metric in a dictionary        
        for image_counter, image_key in enumerate(selected_image_keys):
            (img, img_string) = image_dict[image_key]  # Get input image
    
            ##### Calculate heatmaps for all images for this metric
            # Note that these metrics are univariate, so we only feed in one image and only care about first heatmap returned.
            img_heatmap, dummy_heatmap = compute_metric_locally(img, img, metric = my_metric)
            # store all heatmaps for this metric in a dictionary
            img_row_dict[image_key] = ( img_heatmap, img_string )

        # Prep: calculate uniform cmin and cmax for this row (if desired)
        if want_uniform_color_scale_per_metric:  
            for image_counter, image_key in enumerate(selected_image_keys):
                cmin_uniform, cmax_uniform = value_range_for_image_set( selected_image_keys, img_row_dict )
                
        # For each metric: PLOT all heatmaps
        for image_counter, image_key in enumerate(selected_image_keys):

            # retrieve heatmap for plotting
            (img_heatmap, img_string) = img_row_dict[image_key]
            
            # Determine limits for color bar.  Note: min/max/mean values of heatmap are printed underneath each plot. 
            if ( want_uniform_color_scale_per_metric ):
                (cmin, cmax) = (cmin_uniform, cmax_uniform) 
            else:
                (cmin, cmax) = ( img_heatmap.min(), img_heatmap.max() )
                
            i_row = metric_counter+1
            i_col = image_counter
            axs[i_row][i_col].imshow(img_heatmap, clim=(np.nanmin([0.0,cmin]), cmax), cmap='Greens')
            axs[i_row][i_col].set_title(f' Heatmap - {my_metric}')
            axs[i_row][i_col].set_xlabel(f'Min: {img_heatmap.min():.2f}   Mean: {img_heatmap.mean():.2f}   Max: {img_heatmap.max():.2f} ')

        
    # Show the figure and save to file
    plt.tight_layout()
    plt.show()
    (base_img, base_string) = image_dict[base_image_key]  # Get name of base image for filename
    output_filename = f'OUTPUT/{filename_prefix}_{base_string}_Univariate.png';
    print(f'Saving results to {output_filename}')
    plt.savefig(output_filename)

In [ ]:
def apply_bivariate_metrics( selected_metrics, image_dict, selected_image_keys, base_image_key, filename_prefix ):

    print('\n##############################\n     Bivariate metrics\n')
    print('     min_value and max_value of each heatmap are printed underneath each panel.')
    print('     Note: color map for heatmaps vary greatly.  Go from WHITE = min(0,min_value) to DARK GREEN = (max_value).')
    print('##############################\n\n')

    # Create a figure with subplots: columns represent images, rows represent metrics
    n_cols = len(selected_image_keys)
    n_rows = len(selected_metrics) + 1  # metrics, plus one row for original images
    cm = 1/2.54  # conversion factor from inches (default unit) to centimeters
    fig, axs = plt.subplots(
        ncols=n_cols, nrows= n_rows,  
        figsize= ( n_cols * 4, n_rows * 4),  # per image:  5 in width and height 
        sharex="col", sharey="row"
    )
    
    # Create a figure with subplots: columns represent transformations, rows represent metrics
    #fig, axs = plt.subplots(
    #    #nrows= 1+ len(selected_metrics), ncols=len(selected_image_keys), figsize=(2 * len(selected_metrics), 16 * len(selected_image_keys)), sharex="col", sharey="row"
    #    nrows= 1+ len(selected_metrics), ncols=len(selected_image_keys), figsize=(20, 50), sharex="col", sharey="row"
    #)

    # Use min and max of base image to scale first row of images
    (base_img, base_string) = image_dict[base_image_key]
    value_range_for_plotting = ( np.min(base_img), np.max(base_img) )
    
    for img_counter, img_key in enumerate(selected_image_keys):
    
        # Each transform defines one column
        (img, img_string) = image_dict[img_key]
    
        # First row:  plot original image and transformed images
        axs[0][img_counter].imshow(img, clim = value_range_for_plotting, cmap='gray' )
        axs[0][img_counter].set_title(img_string)
        axs[0][img_counter].set_xlabel(f'Min: {img.min():.2f}   Mean: {img.mean():.2f}   Max: {img.max():.2f} ')
    
        # All other rows:  show heatmaps for different metrics  (each metric gets one row)
        for metric_counter, my_metric in enumerate(selected_metrics, start=1):
            
            img_heatmap = compute_metric_locally(base_img, img, metric = my_metric)
            cmin = np.min( (img_heatmap.min(), 0) )
            cmax = img_heatmap.max()
            axs[metric_counter][img_counter].imshow(img_heatmap, clim=(cmin, cmax), cmap='Greens')
            axs[metric_counter][img_counter].set_title(f' Heatmap - {my_metric}')
            axs[metric_counter][img_counter].set_xlabel(f'Min: {img_heatmap.min():.2f}   Mean: {img_heatmap.mean():.2f}   Max: {img_heatmap.max():.2f} ')
            
    # Show the figure and save to file
    plt.tight_layout()
    plt.show()
    output_filename = f'OUTPUT/{filename_prefix}_{base_string}_Bivariate.png';
    print(f'Saving results to {output_filename}')
    plt.savefig(output_filename)


In [ ]:
########################################################
# Prepare collection of base images to use
########################################################

def prepare_set_of_base_images( ):

    ############ A) LOAD OBSERVED IMAGES ############
    file_name_satellite = '../data/kh_ABI_C13.nc'      # Sample satellite image 
    file_name_radar = '../data/kh_MRMS_REFC.nc'        # Corresponding radar image
    file_name_radar_CNN = '../data/kh_GREMLIN_REFC.nc' # Corresponding CNN estimate of radar (using satellite image as input)
    
    i_sample = 50  # Choose any sample
    img_satellite = load_data(file_name_satellite, i_sample)
    img_RADAR     = load_data(file_name_radar, i_sample)
    img_RADAR_CNN = load_data(file_name_radar_CNN, i_sample)
    
    ############ B) CREATE SYNTHETIC IMAGES ###########
    ############ Gaussian blobs
    # Create six Gaussian blobs with exponentially increasing sigma values - to test the effect of varying sigma
    n_pixels=256
    center_x=40;     center_y=50;    sigma=1;    img1 = gaussian_blob(n_pixels, center_x, center_y, sigma)
    center_x=40;     center_y=80;    sigma=2;    img2 = gaussian_blob(n_pixels, center_x, center_y, sigma)
    center_x=40;     center_y=120;   sigma=4;    img3 = gaussian_blob(n_pixels, center_x, center_y, sigma)
    center_x=40;     center_y=180;   sigma=8;    img4 = gaussian_blob(n_pixels, center_x, center_y, sigma)
    center_x=160;    center_y=50;    sigma=16;   img5 = gaussian_blob(n_pixels, center_x, center_y, sigma)
    center_x=160;    center_y=170;   sigma=32;   img6 = gaussian_blob(n_pixels, center_x, center_y, sigma)
    # final image
    img_synthetic_Gaussian = img1 + img2 + img3 + img4 + img5 + img6 
    
    ############ Sinusoidal grating - to test the effect of gentle transitions
    n_pixels=256
    wave_length_in_pixels=50
    alpha_in_degrees=20
    # final image
    img_synthetic_sinusoidal = sinusoidal_grating(n_pixels, wave_length_in_pixels, alpha_in_degrees)
    
    ############ Single, narrow ridge - to test effect of sharp edges
    n_pixels = 256
    fraction = fraction=0.52
    img_1 = black_white(n_pixels, fraction)
    fraction = fraction=0.48
    img_2 = black_white(n_pixels, fraction)
    #final image
    img_synthetic_ridge = img_2 - img_1
    
    ############ XOR fractal - contains lots of areas with different "texture"
    n_pixels=256
    img_synthetic_fractal = xor_fractal( n_pixels )
    
    # store all images and corresponding text in dictionary to be returned
    image_collection_dict = {
        'satellite'  : (img_satellite, "Satellite"),
        'radar'      : (img_RADAR, "Radar"),
        'radar_CNN'  : (img_RADAR_CNN, "Radar_CNN"),
        'gaussian'   : (img_synthetic_Gaussian, "Gaussian_blobs"),
        'sinusoidal' : (img_synthetic_sinusoidal, "Sinusoidal"),
        'ridge'      : (img_synthetic_ridge, "Ridge"),
        'fractal'    : (img_synthetic_fractal, "Fractal")
    }

    # print all images in collection we created
    for img_counter, (key, (img, img_text)) in enumerate(image_collection_dict.items()):
        plt.figure( img_counter )
        plt.imshow(img, cmap='gray')
        plt.title(img_text)
        plt.xlabel(f'Min: {img.min():.3f}     Mean: {img.mean():.3f}     Max: {img.max():.3f} ')
        plt.show()
    
    return( image_collection_dict )

In [ ]:
#############################################################################
# Define a set of various transformations to be applied to base image to get an idea of how measures differ.
# This is also where parameters are chosen for the different transformations.
#############################################################################

def apply_wide_range_of_transforms_to_image( original_img, original_img_string ):
    
    # Apply transformation 100% of the time (randomness turned off)
    rate = 1
    
    # Apply the Gaussian Blur to the image
    my_transform_str = 'blur' # This works
    blurr_sigma = 1   # Choose parameter
    blurr_string = f'{"Blurred (sigma="}{blurr_sigma:.2f}{")"}'
    blurred_img = GaussianBlur(rate, sigma=blurr_sigma)(original_img)   # Why does this create a modified image even for sigma=0?
    
    # Add Gaussian Noise to image
    my_transform_str = 'noise'
    noise_sigma = 0.2   # Choose parameter
    noise_string = f'{"Noise added (sigma="}{noise_sigma:.2f}{"* max)"}'
    noise_img = GaussianNoise(rate, noise=noise_sigma)(original_img)    # Why does this create a modified image even for noise=0?
    
    # Apply the Brightness adjustment transformation to the image
    my_transform_str = 'brightness' # This works
    brightness_factor = 0.5   # Choose parameter
    brightness_string = f'{"Brightness modified (factor="}{brightness_factor:.2f}{")"}'
    brightness_img = AdjustBrightness(rate, brightness=brightness_factor)(original_img)
    
    # Crop image (TBC) - Lander, can you please fill in?
    my_transform_str = 'crop' 
    # crop parameter
    # crop_img = 
    # cropstring = 
    
    # Apply the H-flip transformation to the image
    my_transform_str = 'hflip' # This works
    hflip_img = RandHorizontalFlip(rate)(original_img)
    hflip_string = "Flipped horizontally"
    
    # Apply the V-flip transformation to the image
    my_transform_str = 'vflip' # This works
    vflip_img = RandVerticalFlip(rate)(original_img)
    vflip_string = "Flipped vertically"
    
    # Store everything in a dictionary
    transformed_image_dict = { 
        'original'   : (original_img, original_img_string), 
        'blur'       : (blurred_img, blurr_string),
        'noise'      : (noise_img, noise_string),
        'brightness' : (brightness_img, brightness_string),
        #'crop'       : (crop_img, crop_string),    # still needs to be added!
        # 'invert'    : (invert_img, invert_string),  # not yet defined.  Useful?
        'hflip'      : (hflip_img, hflip_string), 
        'vflip'      : (vflip_img, vflip_string)
    }

    return( transformed_image_dict )

In [ ]:
####### Step 1: Prepare set of base images and plot them
base_image_dict = prepare_set_of_base_images()

In [ ]:
############ EXPERIMENT 1: ############
############ Apply a bunch of different transformations just to get a first feeling for how the metrics change ############

####### Step 1: Select a single image, select and apply transformations which includes plotting resulting images.
# Select base image: 
# Available choices: [ 'satellite', 'radar', 'radar_CNN', 'gaussian', 'sinusoidal', 'ridge', 'fractal' ]
#base_image_key = 'satellite'

#for base_image_key in [ 'satellite', 'radar', 'radar_CNN', 'gaussian', 'sinusoidal', 'ridge', 'fractal' ]:
for base_image_key in [ 'satellite', 'gaussian' ]:
    
    print(f'\n       Selected image for analysis:     {base_image_key}\n')
    
    # Select transformations to apply
    # Available transforms: [ 'original', 'hflip', 'vflip', 'brightness', 'blur', 'noise' ]
    selected_transforms_keys = [ 'original', 'blur', 'noise', 'brightness', 'hflip', 'vflip' ]
    
    (original_img, original_img_string) = base_image_dict[base_image_key]
    # Generate transformed images
    transformed_image_dict = apply_wide_range_of_transforms_to_image( original_img, original_img_string )
    
    ####### Step 2: Apply selected transforms to origimal image and plot results
    
    # Metrics
    # Available bivariate: ['mse', 'mae', 'rmse', 'psnr', 'ncc', 'grad-ds', 'grad-rmse', 'laplace-rmse', 'hist-int', 'hog-pearson', 'fourier-similarity', 'wavelet-similarity']
    
    ####### Step 2a) Apply univariate metrics and plot heatmaps
    
    # Available univariate metrics: ["mgm", "s1", "tv", "grad-tv", "fourier-tv", "wavelet-tv"]
    selected_metrics_univariate = [ "mgm", "tv", "grad-tv", "s1", "fourier-tv", "wavelet-tv"  ]
    original_image_key = 'original'
    want_uniform_color_scale_per_metric = True
    filename_prefix = 'Exp_1b_GENERIC'
    apply_univariate_metrics( selected_metrics_univariate, transformed_image_dict, selected_transforms_keys, original_image_key, want_uniform_color_scale_per_metric, filename_prefix )
    
    
    ####### Step 2b) Apply bivariate metrics and plot heatmaps
    
    # Available bivariate metrics: ['mse', 'mae', 'rmse', 'psnr', 'ncc', 'grad-ds', 'grad-rmse', 'laplace-rmse', 'hist-int', 'hog-pearson', 'fourier-similarity', 'wavelet-similarity']
    # Lander - Let's add SSIM
    #bi_metrics_set_1 = ['mse', 'mae', 'rmse', 'psnr', 'ncc'] # Difference in pixel space 
    #bi_metrics_set_2 = ['hist-int', 'hog-pearson'] # Difference in histogram space?
    #bi_metrics_set_3 = ['grad-ds', 'grad-rmse', 'laplace-rmse']   # Difference in gradient space
    #bi_metrics_set_4 = ['fourier-similarity', 'wavelet-similarity'] # Difference in spectral space
    selected_metrics_bivariate = ['mse', 'mae', 'rmse', 'psnr', 'ncc', 'grad-ds', 'grad-rmse', 'laplace-rmse', 'hist-int', 'hog-pearson', 'fourier-similarity', 'wavelet-similarity']
    # Removed metric 'fourier-similarity' temporarily, so that it doesn't throw an error until fixed.
    selected_metrics_bivariate = ['mse', 'mae', 'rmse', 'psnr', 'ncc', 'grad-ds', 'grad-rmse', 'laplace-rmse', 'hist-int', 'hog-pearson', 'wavelet-similarity']

    original_image_key = 'original'
    filename_prefix = 'Exp_1b_GENERIC_'
    apply_bivariate_metrics( selected_metrics_bivariate, transformed_image_dict, selected_transforms_keys, original_image_key, filename_prefix )


In [ ]:
############ EXPERIMENT 2: ############
############ COMPARE Observed radar to CNN-generated radar image (from GREMLIN) ############

############ LOAD IMAGES ############
file_name_satellite = '../data/kh_ABI_C13.nc'      # Sample satellite image 
file_name_radar = '../data/kh_MRMS_REFC.nc'        # Corresponding radar image
file_name_radar_CNN = '../data/kh_GREMLIN_REFC.nc' # Corresponding CNN estimate of radar (using satellite image as input)

selected_samples = range( 0, 200, 50 )
#selected_samples = range( 0, 100, 20 )

for i_sample in selected_samples:
    img_satellite = load_data(file_name_satellite, i_sample)
    img_RADAR     = load_data(file_name_radar, i_sample)
    img_RADAR_CNN = load_data(file_name_radar_CNN, i_sample)
    
    radar_image_dict = {
        # 'satellite'  : (img_satellite, "Satellite"),
        #radar_text = print( f'Radar
        'radar'      : (img_RADAR, f'Radar_sample_{i_sample}'),
        'radar_CNN'  : (img_RADAR_CNN, f'Radar_CNN_sample_{i_sample}'),
    }

    selected_metrics_univariate = [ "s1", "mgm", "tv", "grad-tv", "fourier-tv", "wavelet-tv" ]
    #selected_metrics_univariate = [ "mgm", "tv", "grad-tv", "fourier-tv", "wavelet-tv", "s1" ]
    selected_image_keys = [ 'radar', 'radar_CNN' ]
    original_image_key = 'radar'
    want_uniform_color_scale_per_metric = True
    filename_prefix = 'Exp_2_GREMLIN'

    apply_univariate_metrics( selected_metrics_univariate, radar_image_dict, selected_image_keys, original_image_key, want_uniform_color_scale_per_metric, filename_prefix )

    selected_metrics_bivariate = ['mse', 'mae', 'rmse', 'psnr', 'ncc', 'grad-ds', 'grad-rmse', 'laplace-rmse', 'hist-int', 'hog-pearson', 'fourier-similarity', 'wavelet-similarity']
    # Removing fourier-similarity temporarily
    selected_metrics_bivariate = ['mse', 'mae', 'rmse', 'psnr', 'ncc', 'grad-ds', 'grad-rmse', 'laplace-rmse', 'hist-int', 'hog-pearson', 'wavelet-similarity']
    original_image_key = 'radar'
    filename_prefix = 'Exp_2_GREMLIN'
    apply_bivariate_metrics( selected_metrics_bivariate, radar_image_dict, selected_image_keys, original_image_key, filename_prefix )

In [ ]:
# Ignore these last two cells.  Not ready to be used yet!

#############################################################################
# Define a set of noise transformations to be applied to base image.
#############################################################################

'''
def apply_varying_amount_of_noise_to_image( original_img, original_img_string ):
    
    # Apply transformation 100% of the time (randomness turned off)
    rate = 1

    selected_sigmas = np.arange(0.0, 1.0, 0.1)   #range( 0.0, 1.0, 0.1 )

    transformed_image_dict = { }
    
    for noise_sigma in selected_sigmas:
        
        # Add Gaussian Noise to image
        my_transform_str = 'noise'
        noise_sigma = 0.2   # Choose parameter
        noise_string = f'{"Noise added (sigma="}{noise_sigma:.2f}{"* max)"}'
        noise_img = GaussianNoise(rate, noise=noise_sigma)(original_img)    # Why does this create a modified image even for noise=0?

        # Add image to dictionary
        key_string = f'noise_{noise_sigma:.1f}'
        print(my_string)
        transformed_image_dict[key_string] = ( noise_img , noise_string )
        

    return( transformed_image_dict )
'''

In [ ]:
# Ignore these last two cells.  Not ready to be used yet!


############ EXPERIMENT 3: ############
############ Use different amounts of noise ############

'''
# Available choices: [ 'satellite', 'radar', 'radar_CNN', 'gaussian', 'sinusoidal', 'ridge', 'fractal' ]

# Select base image: 
# Available choices: [ 'satellite', 'radar', 'radar_CNN', 'gaussian', 'sinusoidal', 'ridge', 'fractal' ]
base_image_key = 'satellite'
(original_img, original_img_string) = base_image_dict[base_image_key]

apply_varying_amount_of_noise_to_image( original_img, original_img_string )
'''
